In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')
!pip install gensim
import gensim
from gensim.utils import simple_preprocess
!pip install tqdm
import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [ ]:
df=pd.read_csv('balanced_ai_human_prompts.csv')
df.head()

,text,generated
0,"Machine learning, a subset of artificial intel...",1
1,"A decision tree, a prominent machine learning ...",1
2,"Education, a cornerstone of societal progress,...",1
3,"Computers, the backbone of modern technology, ...",1
4,"Chess, a timeless game of strategy and intelle...",1


In [ ]:
df.duplicated().sum()

330

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

0

In [ ]:
df['generated'].value_counts()

,count
generated,
0,1375
1,1045


In [ ]:
df.isnull().sum()

,0
text,0
generated,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2420 entries, 0 to 2749
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       2420 non-null   object
 1   generated  2420 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 56.7+ KB


In [ ]:
#removing special characters
df['text']=df['text'].apply(lambda x: re.sub('[^a-z A-Z 0-9]+','',x))
#removing stopwords
df['text']=df['text'].apply(lambda x: " ".join([y for y in x.split() if y not in set(stopwords.words('english'))]))
#removing urls
df['text']=df['text'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&/~+#-]*[\w@?^=%&/~+#])?','',str(x)))
#removing html tags
df['text']=df['text'].apply(lambda x: BeautifulSoup(x,'lxml').get_text())
#removing additional spaces
df['text']=df['text'].apply(lambda x: " ".join(x.split()))

In [ ]:
df['text']=df['text'].str.lower()

In [ ]:
lemmatizer=WordNetLemmatizer()

In [ ]:
def lemmatize_word(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [ ]:
df['text']=df['text'].apply(lambda x: lemmatize_word(x))

In [ ]:
words=[]
for i in df.index:
    sent_token=sent_tokenize(df['text'][i])
    for j in sent_token:
        words.append(simple_preprocess(j))

In [ ]:
len(words)

2420

In [ ]:
model=gensim.models.Word2Vec(words)

In [ ]:
model.wv.index_to_key

['car',
 'vote',
 'electoral',
 'people',
 'college',
 'state',
 'the',
 'would',
 'president',
 'elector',
 'candidate',
 'one',
 'election',
 'usage',
 'many',
 'popular',
 'city',
 'in',
 'get',
 'way',
 'day',
 'this',
 'also',
 'even',
 'pollution',
 'system',
 'voter',
 'driving',
 'less',
 'it',
 'voting',
 'source',
 'use',
 'world',
 'smog',
 'air',
 'make',
 'could',
 'time',
 'go',
 'like',
 'much',
 'gas',
 'limiting',
 'country',
 'percent',
 'dont',
 'say',
 'want',
 'take',
 'life',
 'transportation',
 'united',
 'environment',
 'place',
 'citizen',
 'help',
 'paris',
 'may',
 'without',
 'if',
 'need',
 'thing',
 'new',
 'reason',
 'change',
 'amount',
 'emission',
 'advantage',
 'context',
 'think',
 'know',
 'good',
 'better',
 'stress',
 'ban',
 'unfair',
 'traffic',
 'year',
 'around',
 'problem',
 'process',
 'always',
 'see',
 'money',
 'work',
 'greenhouse',
 'there',
 'public',
 'america',
 'mean',
 'using',
 'going',
 'due',
 'article',
 'bogota',
 'example',
 

In [ ]:
model.corpus_count

2420

In [ ]:
model.epochs

5

In [ ]:
def avg_word2vec(doc):
    words_in_vocab=[word for word in doc if word in model.wv.index_to_key]
    if not words_in_vocab:
        return np.zeros(model.wv.vector_size)
    return np.mean([model.wv[word] for word in words_in_vocab],axis=0)

In [ ]:
X=[]
for i in tqdm.tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

100%|██████████| 2420/2420 [00:07<00:00, 327.81it/s] 


In [ ]:
X1=np.array(X)

In [ ]:
df1=pd.DataFrame(X1)
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.121508,0.181537,0.002442,-0.045969,-0.077549,-0.268128,0.081779,0.462837,0.112118,-0.220317,...,0.021336,-0.107667,0.186709,0.082578,0.165187,0.023531,-0.024203,-0.108464,0.121282,-0.024669
1,-0.016027,0.337327,0.072321,0.161657,-0.103542,-0.189061,0.078888,0.473663,-0.155075,-0.246709,...,0.127345,-0.015345,0.049914,0.034284,0.330659,0.105082,0.153442,-0.005630,0.151629,0.047237
2,0.111047,0.198792,0.006603,0.009411,-0.041300,-0.297606,0.084076,0.408456,0.088288,-0.221338,...,-0.000764,-0.060221,0.170826,0.074967,0.240641,0.060841,-0.065005,-0.136093,0.090640,-0.007927
3,0.242994,0.129418,0.018652,-0.040535,-0.049035,-0.430646,0.081826,0.419807,0.195987,-0.203934,...,-0.041805,-0.061288,0.229777,0.117821,0.191709,0.002830,-0.168400,-0.298853,0.087589,0.013076
4,0.177460,0.171782,0.036286,-0.016115,-0.054308,-0.284820,0.103141,0.391780,0.072303,-0.187706,...,0.008546,-0.075801,0.133607,0.064768,0.190662,0.054372,-0.009053,-0.145848,0.100494,0.011197


In [ ]:
# Align indices before assigning the 'generated' column
df1['rating'] = df['generated'].reset_index(drop=True)

In [ ]:
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,rating
0,0.121508,0.181537,0.002442,-0.045969,-0.077549,-0.268128,0.081779,0.462837,0.112118,-0.220317,...,-0.107667,0.186709,0.082578,0.165187,0.023531,-0.024203,-0.108464,0.121282,-0.024669,1
1,-0.016027,0.337327,0.072321,0.161657,-0.103542,-0.189061,0.078888,0.473663,-0.155075,-0.246709,...,-0.015345,0.049914,0.034284,0.330659,0.105082,0.153442,-0.005630,0.151629,0.047237,1
2,0.111047,0.198792,0.006603,0.009411,-0.041300,-0.297606,0.084076,0.408456,0.088288,-0.221338,...,-0.060221,0.170826,0.074967,0.240641,0.060841,-0.065005,-0.136093,0.090640,-0.007927,1
3,0.242994,0.129418,0.018652,-0.040535,-0.049035,-0.430646,0.081826,0.419807,0.195987,-0.203934,...,-0.061288,0.229777,0.117821,0.191709,0.002830,-0.168400,-0.298853,0.087589,0.013076,1
4,0.177460,0.171782,0.036286,-0.016115,-0.054308,-0.284820,0.103141,0.391780,0.072303,-0.187706,...,-0.075801,0.133607,0.064768,0.190662,0.054372,-0.009053,-0.145848,0.100494,0.011197,1


In [ ]:
X=df1.drop('rating',axis=1)
y=df1['rating']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [ ]:
lg=LogisticRegression()
lg.fit(X_train,y_train)
y_pred=lg.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.996694214876033


In [ ]:
sv=SVC()
sv=sv.fit(X_train,y_train)
y_pred=sv.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.996694214876033
